In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import date
from IPython.core import display as ICD


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold

from sklearn.preprocessing import normalize

from sklearn.metrics import mean_squared_error

from math import sqrt
import statistics 


%matplotlib inline 

In [ ]:
data_list = ['transactions', 'items', 'item_categories', 'shops', 'test']
for data in data_list:
    exec(f'{data} '+'= pd.read_csv(data+".csv")')

In [ ]:
data_list = [transactions, items, item_categories, shops, test]
for data in data_list:
    #для печати нескольких фреймов pandas в формате HTML
    ICD.display(data.head())

In [ ]:
transactions['date'] = pd.to_datetime(transactions['date'], format='%d.%m.%Y')
transactions['date'] = transactions.date.dt.month

In [ ]:
transactions[['item_price','item_cnt_day']].describe()

In [ ]:
sns.jointplot(x='item_cnt_day', y='item_price', data=transactions, height=8)
plt.show()

Наблюдается аномальное количество продаж одного товара за один день, удалим этот объект

In [ ]:
# удалям объект с аномальным количеством продаж
index_del = transactions.loc[transactions['item_cnt_day']>50].index
transactions.drop(transactions.index[index_del], inplace=True)

Наблюдается отрицательная цена, скорее всего это ошибка, удаляем эту строку

In [ ]:
# удалям объекты с отрицательной ценой
index_del = transactions.loc[(transactions['item_price']<0)&(transactions['item_price']>75000)].index
transactions.drop(transactions.index[index_del], inplace=True)

Проверим все ли item_id тренировочной выборки есть в тестовой

In [ ]:
item_id_test = test['item_id'].unique().tolist()
print (f'Колличество уникальных item_id в тестовой выборке: {len(item_id_test)}')

In [ ]:
item_id_train = transactions['item_id'].unique().tolist()
print (f'Колличество уникальных item_id в тренировочной выборке: {len(item_id_train)}')

In [ ]:
item_id_test_train = list(set(item_id_train) & set(item_id_test))
print (f'Колличество item_id которые есть в тестовой и тренировочной выборках: {len(item_id_test_train)}')

Таким образом, кроме того ,что в тестовой выборке не все  item_id из тренировочной, в ней ещё присутствуют item_id, которых нет в тренировочной, заменим количество продаж таких объектов на 0

In [ ]:
transactions_new = transactions[transactions['item_id'].isin(item_id_test_train)]

In [ ]:
transactions_new[['item_price','item_cnt_day']].describe()

In [ ]:
transactions_new['item_cnt_day'].plot(kind='box')

plt.title('Item price')
plt.ylabel('Item price')

plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = transactions_new['item_price'].values
X = X.reshape(-1,1)
scaler.fit(X)
scaler_X = scaler.transform(X)
transactions_new['item_price_minmax'] = scaler_X.copy() 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = transactions_new['item_price'].values
X = X.reshape(-1,1)
scaler.fit(X)
scaler_X = scaler.transform(X)
transactions_new['item_price_std'] = scaler_X 

In [ ]:
# группировка продаж по месяцам и суммирование всех продаж всех магазинов
transactions_group = transactions_new.groupby(['date_block_num'])['item_cnt_day'].sum().to_frame().reset_index()
transactions_group.head()

In [ ]:
# График продаж всех товаров во всех магазинах втечении всего периода
plt.plot(transactions_group['date_block_num'], transactions_group['item_cnt_day'],'y')
plt.ylabel('Num items')
plt.xlabel('Month')
plt.title("Month item counts  for all shops")
plt.show()

In [ ]:
# группировка по месяцу-магазину-товару с заменой цены и ежедневных продаж на средние
transactions_mean = transactions_new.groupby(['date_block_num','shop_id','item_id']).mean().reset_index()

In [ ]:
# группировка по месяцу-магазину-товару с заменой цены и ежедневных продаж на сумму
transactions_sum = transactions_new.groupby(['date_block_num','shop_id','item_id']).sum().reset_index()

In [ ]:
# создание нового фрейма количество продаж сгруппированных по месяцу-магазину-товару с среденей ценой товара в месяце
transactions_new = transactions_mean.copy()
transactions_new['item_cnt_day'] = transactions_sum['item_cnt_day'].values
transactions_new.rename(columns={'item_cnt_day': 'target'}, inplace=True)

In [ ]:
print(f'Количество уникальных item_id в данных: {len(transactions_new["item_id"].unique().tolist())}')

Количество уникальных item_id в преабразованных данных 4737, в тестовой выборке 5100, добавим пары shop_id-item_id и ID из тестовой выборки

In [ ]:
transactions_new.set_index(['shop_id','item_id'], inplace = True)
# создаем данные для нулевого месяца
transactions_result = transactions_new.loc[transactions_new['date_block_num'] == 0]
# присоеденяем ID из тестовых данных
transactions_result = transactions_result.join(test.set_index(['shop_id','item_id']), how = 'outer')
# заполняем date_block_num сответствующим номером месяца
transactions_result['date_block_num'].fillna(0, inplace=True)


# все тоже самое для всех оставшихся месяцев (с 0 поступил так, чтобы не мучаться с pd.concat)
for month in range(1,34):
    transactions_month = transactions_new.loc[transactions_new['date_block_num'] == month]
    # присоеденяем ID из тестовых данных
    transactions_month = transactions_month.join(test.set_index(['shop_id','item_id']), how = 'outer')
    # заполняем date_block_num сответствующим номером месяца
    transactions_month['date_block_num'].fillna(month, inplace=True) 
    # присоеденяем очередной месяц
    transactions_result = pd.concat([transactions_result, transactions_month])

In [ ]:
# функция создания цен для пропущенных столбцов
# после добавления shop_id-item_id из тестового набора
def make_price (df, column_list):
    for column in column_list:
        column = str(column)
        new_column = column+'_tmp'
        # формируем столбец с средними значениями цен по товарам за весь период
        df[new_column] = df.groupby('item_id')[column].transform('mean')
        
    for column in column_list:
        column = str(column)
        new_column = column+'_tmp_month'
        # формируем столбец с средними значениями цен по товарам по месяцам
        df[new_column] = df.groupby(['date_block_num','item_id'])[column].transform('mean')
    return

# функция заполнения цен для пропущенных столбцов
# после добавления shop_id-item_id из тестового набора
def fill_price (df, column_list):
    for column in column_list:
        column = str(column)
        # заполняем отсутсвующие значения цен средними по месяцам (если есть)
        df[column].fillna(df[column+'_tmp_month'], inplace=True)
        # удаляем временный столбец
        df.drop([column+'_tmp_month'], axis = 1, inplace = True)

    for column in column_list:
        column = str(column)
        # заполняем отсутсвующие значения цен средними за весь период
        df[column].fillna(df[column+'_tmp'], inplace=True)
        # удаляем временный столбец
        df.drop([column+'_tmp'], axis = 1, inplace = True)
    # удаляем объекты, которых нет в тестовом наборе
    df.dropna(subset = ['ID'], inplace = True)
    # если в соответсвующем месяце не было продаж, запоняем 0 (это все NaN которые остались)
    df.fillna(0, inplace=True)
    return df.head()

In [ ]:
transactions_result['flag_miss_price'] = pd.isna(transactions_result['item_price'])
transactions_result['flag_miss_price'] = np.where(transactions_result['flag_miss_price'], 1, 0)

In [ ]:
# применяем созданные функции
column_list = ['item_price','item_price_minmax','item_price_std']
make_price (transactions_result, column_list)
fill_price (transactions_result, column_list)
transactions_result.reset_index(inplace = True)

In [ ]:
# добавляем категорийный признак: категория товара, суб-категория товара, городской код
transactions_result['item_categories_id'] = transactions_result['item_id'].map(items['item_category_id'])
transactions_result['item_categories'] = transactions_result['item_categories_id'].map(item_categories['item_category_new'])
transactions_result['sub_categories'] = transactions_result['item_categories_id'].map(item_categories['sub_cat'])
transactions_result['city_code'] = transactions_result['shop_id'].map(shops['city_code'])
transactions_result.drop(columns = 'item_categories_id', inplace = True)

In [ ]:
transactions_result.shape

In [ ]:
# сортируем как в тестовом наборе
transactions_result = transactions_result.sort_values(by=['date_block_num','ID'])
transactions_result.reset_index(inplace = True)
transactions_result.drop(['index'], axis = 1, inplace = True)

In [ ]:
# создаем тестовые данные с добавлением средних цен
transactions_result_33 = transactions_result.loc[transactions_result['date_block_num']<34]
test = transactions_result_33.groupby(['ID','shop_id','item_id'])[['date_block_num','item_price','target','item_price_minmax','item_price_std']].mean()
test['target'] = 0
test['date_block_num'] = 34
test['flag_miss_price'] = 1
test['date'] = 11
test.reset_index(inplace = True)

In [ ]:
# добавляем категорийный признак: категория товара
test['item_categories_id'] = test['item_id'].map(items['item_category_id'])
test['item_categories'] = test['item_categories_id'].map(item_categories['item_category_new'])
test['sub_categories'] = test['item_categories_id'].map(item_categories['sub_cat'])
test['city_code'] = test['shop_id'].map(shops['city_code'])
test.drop(columns = 'item_categories_id', inplace = True)

In [ ]:
test = test.sort_values(by=['date_block_num','ID'])

In [ ]:
# создаем одинаковый порядок коллонок
columns = test.columns.tolist()
transactions_result = transactions_result[columns]

In [ ]:
transactions_result = pd.concat([transactions_result,test])

In [ ]:
# создаем данные для mean-encodings
mean_target_df = transactions_result[['ID','shop_id','item_id','date_block_num','target']].copy()

In [ ]:
# mean-encodings
kf = KFold(n_splits = 5, shuffle=False)

train, val = zip(*kf.split(mean_target_df))
for train_index, val_index, i in zip(train, val, range(5)):
    
    item_id_target_mean = mean_target_df.iloc[train_index].groupby('item_id').target.mean()
    mean_target = mean_target_df.iloc[val_index]
    mean_target['item_target_enc'] = mean_target['item_id'].map(item_id_target_mean)
    mean_target['item_target_enc'].fillna(0.3343, inplace=True) 
    if i == 0:
        mean_target_result = mean_target.copy()
    else:
        mean_target_result = pd.concat([mean_target_result,mean_target])
mean_target_df = mean_target_result.sort_values(by=['date_block_num','ID'])  
#encoded_feature = all_data_result['item_target_enc'].values

In [ ]:
transactions_result['item_target_enc'] = mean_target_df['item_target_enc'].values
del mean_target_df

In [ ]:
def down (df, column_list = []):
    not_int_columns = column_list
    int_columns = list(df.columns.difference(not_int_columns))
    for col in int_columns:
        df[col] = df[col].astype('int32')
    for col in not_int_columns:
        df[col] = df[col].astype('float32')

In [ ]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df.head()

In [ ]:
down (transactions_result)

In [ ]:
downcast_dtypes(transactions_result)

In [ ]:
transactions_group = transactions_result.groupby(['shop_id','date_block_num'])['target'].sum().to_frame()

In [ ]:
table = pd.pivot_table(transactions_group, values='target', index=['shop_id'],
                    columns=['date_block_num'])
table.drop([34], axis = 1, inplace = True)

In [ ]:
colum = table.columns
for column in colum:
    table.rename(columns={column: 'data'+str(column)}, inplace=True)

In [ ]:
group_items_shop = transactions_result.groupby(['shop_id','item_id'])['target'].sum().to_frame()

In [ ]:
table_items_shop = pd.pivot_table(group_items_shop, values='target', index=['shop_id'],
                    columns=['item_id'])

In [ ]:
table = pd.concat([table, table_items_shop], axis = 1)

In [ ]:
del table_items_shop

In [ ]:
X = table.values
X = normalize(X)

kmeans = KMeans(n_clusters=2, random_state=241)
kmeans.fit(X)
labels = pd.DataFrame(kmeans.labels_, columns = ['labels'])

In [ ]:
labels['labels'].value_counts()

In [ ]:
table['cluster_shop'] = kmeans.labels_
map_shop_cluster = table['cluster_shop']

In [ ]:
transactions_result['cluster_shop'] = transactions_result['shop_id'].map(map_shop_cluster)
downcast_dtypes(transactions_result)

In [ ]:
transactions_group_item = transactions_result.groupby(['item_id','date_block_num'])['target'].sum().to_frame()

In [ ]:
table_item = pd.pivot_table(transactions_group_item, values='target', index=['item_id'],
                    columns=['date_block_num'], margins_name = ['date_block_num'])
table_item.drop([34], axis = 1, inplace = True)

In [ ]:
item_shop = transactions_result.groupby(['item_id','shop_id'])['target'].sum().to_frame()

In [ ]:
table_item_shop = pd.pivot_table(item_shop, values='target', index=['item_id'],
                    columns=['shop_id'])

In [ ]:
colum = table_item_shop.columns

In [ ]:
for column in colum:
    table_item_shop.rename(columns={column: 'id'+str(column)}, inplace=True)

In [ ]:
item_price = transactions_result.groupby(['item_id','date_block_num'])['item_price_minmax'].mean().to_frame()

In [ ]:
table_item_price = pd.pivot_table(item_price, values='item_price_minmax', index=['item_id'],
                    columns=['date_block_num'])
table_item_price.drop([34], axis = 1, inplace = True)

In [ ]:
colum = table_item_price.columns
for column in colum:
    table_item_price.rename(columns={column: 'data'+str(column)}, inplace=True)

In [ ]:
table_item = pd.concat([table_item, table_item_shop, table_item_price], axis = 1)

In [ ]:
del table_item_shop
del table_item_price

In [ ]:
X = table_item.iloc[:,1:]
X = normalize(X)

kmeans = KMeans(n_clusters=3, random_state=241)
kmeans.fit(X)
labels = pd.DataFrame(kmeans.labels_, columns = ['labels'])

In [ ]:
labels['labels'].value_counts()

In [ ]:
table_item['cluster_item'] = kmeans.labels_

In [ ]:
map_cluster = table_item['cluster_item']

In [ ]:
transactions_result['cluster_item'] = transactions_result['item_id'].map(map_cluster)
downcast_dtypes(transactions_result)

In [ ]:
# признак колличество продаж магазин-категория_товара-месяц
target_shop_categories = transactions_result.groupby(['shop_id', 'item_categories', 'date_block_num'],as_index=False).agg({'target':{'target_shop_categories':'sum'}})
target_shop_categories.columns = [col[0] if col[-1]=='' else col[-1] for col in target_shop_categories.columns.values]
# признак колличество продаж магазин-месяц
target_shop = transactions_result.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'target':{'target_shop':'sum'}})
target_shop.columns = [col[0] if col[-1]=='' else col[-1] for col in target_shop.columns.values] # этот код для того чтобы название target_shop не было вместе с target
# признак колличество продаж товар-месяц
target_item = transactions_result.groupby(['item_id', 'date_block_num'],as_index=False).agg({'target':{'target_item':'sum'}})
target_item.columns = [col[0] if col[-1]=='' else col[-1] for col in target_item.columns.values]
# признак колличество продаж категория-месяц
target_item_categories = transactions_result.groupby(['item_categories', 'date_block_num'],as_index=False).agg({'target':{'target_item_categories':'sum'}})
target_item_categories.columns = [col[0] if col[-1]=='' else col[-1] for col in target_item_categories.columns.values]

In [ ]:
# приклеиваем новые признаки к данным
transactions_result = transactions_result.set_index(['shop_id', 'item_categories', 'date_block_num']).join(target_shop_categories.set_index(['shop_id', 'item_categories', 'date_block_num']))
transactions_result.reset_index(inplace = True)
#del target_shop_categories
transactions_result = transactions_result.set_index(['shop_id','date_block_num']).join(target_shop.set_index(['shop_id','date_block_num']))
transactions_result.reset_index(inplace = True)
#del target_shop
transactions_result = transactions_result.set_index(['item_id','date_block_num']).join(target_item.set_index(['item_id','date_block_num']))
transactions_result.reset_index(inplace = True)
#del target_item
transactions_result = transactions_result.set_index(['item_categories','date_block_num']).join(target_item_categories.set_index(['item_categories','date_block_num']))
transactions_result.reset_index(inplace = True)
#del target_item_categories

In [ ]:
# сортируем как в тестовом наборе
transactions_result = transactions_result.sort_values(by=['date_block_num','ID'])
transactions_result.reset_index(inplace = True)
transactions_result.drop(['index'], axis = 1, inplace = True)

In [ ]:
transactions_result_tmp = transactions_result[['ID','shop_id', 'item_id', 'date_block_num','target','target_shop','target_item','target_item_categories',]].copy()

In [ ]:
downcast_dtypes(transactions_result_tmp)

In [ ]:
index_cols = ['ID','shop_id', 'item_id', 'date_block_num']
# List of columns that we will use to create lags

cols_to_rename = list(transactions_result_tmp.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 6, 12, 13, 14, 15, 16, 17]

for month_shift in shift_range:
    
    print(month_shift)
    
    train_shift = transactions_result_tmp[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    transactions_result_tmp = pd.merge(transactions_result_tmp, train_shift, on=index_cols, how='left')

del train_shift

In [ ]:
for col in transactions_result_tmp.columns.to_list():
    transactions_result_tmp[col].fillna(0, inplace = True)

In [ ]:
down (transactions_result_tmp)

In [ ]:
transactions_result.reset_index(inplace = True)
transactions_result.drop(['index'], axis = 1, inplace = True)

In [ ]:
downcast_dtypes(transactions_result)

In [ ]:
for miss in ['target_shop_categories','target_shop','target_item','target_item_categories']:
    miss = miss + "_miss"
    transactions_result[miss] = 0
    transactions_result.loc[transactions_result['date_block_num']==34, miss] = 1

In [ ]:
transactions_result = pd.concat([transactions_result, transactions_result_tmp.iloc[:,8:]], axis = 1)

In [ ]:
transactions_result.reset_index().head()

In [ ]:
downcast_dtypes(transactions_result)

In [ ]:
transactions_result['item_info'] = transactions_result['item_id'].map(items['item_name_translated'])

In [ ]:
transactions_result.info()

In [ ]:
transactions_result.to_csv('transactions_result.csv', index = False)